In [ ]:
[1]*10**10

Here we import the relative packages.


In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

!pip install tensorflow_text
!pip install transformers

import os
import tensorflow as tf
import tensorflow_hub as hub

import tensorflow.keras as keras

import tensorflow_text as text

from transformers import LongformerTokenizerFast, TFLongformerModel


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached tensorflow_text-2.9.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.6 MB)
  Using cached tensorflow-2.9.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (511.7 MB)
  Using cached tensorboard-2.9.0-py3-none-any.whl (5.8 MB)
  Using cached keras-2.9.0-py2.py3-none-any.whl (1.6 MB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached tensorflow_estimator-2.9.0-py2.py3-none-any.whl (438 kB)
  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting un

In [ ]:
from transformers import LongformerTokenizerFast

In [ ]:
from transformers import LongformerTokenizer, LongformerForTokenClassification
import torch

tokenizer = LongformerTokenizerFast.from_pretrained("allenai/longformer-base-4096")
model = LongformerForTokenClassification.from_pretrained("allenai/longformer-base-4096")

inputs = tokenizer(
    "HuggingFace is a company based in Paris and New York", add_special_tokens=False, return_tensors="pt"
)

#with torch.no_grad():
   # logits = model(**inputs).logits

#predicted_token_class_ids = logits.argmax(-1)


# Note that tokens are classified rather then input words which means that
# there might be more predicted token classes than words.
# Multiple token classes might account for the same word
#predicted_tokens_classes = [model.config.id2label[t.item()] for t in predicted_token_class_ids[0]]
#predicted_tokens_classes

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/694 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570M [00:00<?, ?B/s]

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForTokenClassification: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing LongformerForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerForTokenClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN

Here we mount google drive where we have saved the relevant data and weights for our model.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')





Mounted at /content/drive


In [ ]:
data = pd.read_csv("/content/drive/MyDrive/train.csv")

documents = data.id.unique()
doc_id_map = {}





In [ ]:
!unzip /content/drive/MyDrive/train.zip


Streaming output truncated to the last 5000 lines.
  inflating: train/A34F66AA1F99.txt  
  inflating: train/0DD93F825FD7.txt  
  inflating: train/21C84DF6129A.txt  
  inflating: train/AA0F49E35E96.txt  
  inflating: train/29A8E6DA6F04.txt  
  inflating: train/ED0C40B8C17A.txt  
  inflating: train/F576B3F624B3.txt  
  inflating: train/2F5E6E85679A.txt  
  inflating: train/F5428139729A.txt  
  inflating: train/9ACDEAE29302.txt  
  inflating: train/7A0493F083E6.txt  
  inflating: train/3FF3A8F8AE90.txt  
  inflating: train/C9962F63C98A.txt  
  inflating: train/0EF6C038DD32.txt  
  inflating: train/B064654D4BB8.txt  
  inflating: train/980850873AB7.txt  
  inflating: train/BFEA78A024A9.txt  
  inflating: train/6BC2B4F5FC00.txt  
  inflating: train/3F65AF00FC67.txt  
  inflating: train/ACBB0618CF1A.txt  
  inflating: train/5E75902533E2.txt  
  inflating: train/E38FA6177A9D.txt  
  inflating: train/600959D8A343.txt  
  inflating: train/CC2697CF981F.txt  
  inflating: train/6915155109BD.txt  

In [ ]:
data.head()

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59
2,423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75
3,423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...
4,423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...


In [ ]:
doc_labels= {}
doc_tokens= {}


Here we write a function which tokenizes the training data and converts the training labels to a format which our model can train against.

In [ ]:


## label a token index as belonging to a lead if its image under the offset mapping
## discourse_start<=offset_mapping[index][0]
## offset_mapping[index][1]< discourse_end
## claims_array is an array with the same number of entries as tokens in the text



discourse_elements = {'Lead': 0, 'Position': 1, 'Evidence': 2, 'Claim': 3, 'Concluding Statement': 4, 'Counterclaim': 5, 'Rebuttal': 6}

def get_tokens(string):
  tokens = tokenizer.encode_plus(string, padding= 'max_length',truncation= True, max_length=1000, return_attention_mask= True ,return_offsets_mapping= True, return_token_type_ids = True)
  offset_mapping = tokens.offset_mapping
  return tokens


def get_document_token_labels(document):
  #document dataframe
  
  df = data.loc[data.id == document]

  ## get document text
  with open('train/'+ str(document) +'.txt') as f:
    lines = f.read()
  
  ## tokenize the document 
  tokens = tokenizer.encode_plus(lines, padding= 'max_length',truncation= True, max_length=1000, return_attention_mask= True ,return_offsets_mapping= True, return_token_type_ids = True)
  doc_tokens[document] = tokens
  offset_mapping = tokens.offset_mapping


  # number of tokens

  number_of_tokens = len(tokens['input_ids'])

  ## initialize np arrays for labels

  doc_labels[document] = np.zeros((number_of_tokens,len(discourse_elements)))

  # for loop through discourse elements
  for discourse_element in discourse_elements.keys():
  
    df_discourse = df.loc[df.discourse_type == discourse_element]


    df_count = 0

    offset_mapping = tokens.offset_mapping
    attention = tokens.attention_mask

    discourse_number = discourse_elements[discourse_element]

    ## for loop through each array
    for index in range(number_of_tokens):
      if df_count< len(df_discourse):
        ## dont do anything on padded values
        if attention[index]==0:
          continue
        #print(df_discourse['discourse_start'])
        if offset_mapping[index][1]<df_discourse['discourse_start'].iloc[0]:
        ## dont do anything until you hit the first discourse element
          continue
          
        if df_discourse['discourse_start'].iloc[df_count]<=offset_mapping[index][0] and offset_mapping[index][1]< df_discourse['discourse_end'].iloc[df_count]:
          doc_labels[document][index, discourse_number] = 1

          #move to the next row in the dataframe once you reach the end of a discourse element
          if offset_mapping[index][1] == df_discourse['discourse_end'].iloc[df_count]-1:
            df_count+=1


  return (np.array([doc_tokens[document]['input_ids'],doc_tokens[document]['attention_mask']]), doc_labels[document])


    

This function is optional and prints the labeled labeled tokens from the training data.

In [ ]:
## print labels 

def print_labeled_tokens(document, discourse_element):
  get_document_token_labels(document)
  with open('train/'+ document +'.txt') as f:
    lines = f.read()
  offset_mapping = doc_tokens[document].offset_mapping
  for i, value in enumerate(doc_labels[document][discourse_elements[discourse_element]]):
    if value==1:
      print(lines[offset_mapping[i][0]:offset_mapping[i][1]])



In [ ]:
print_labeled_tokens('423A1CA112E2', 'Position')

Here we download the pretrained longformer and set trainable to false so that during the training of our neural network the longformers weights remain fixed.

In [ ]:
longformer = TFLongformerModel.from_pretrained('allenai/longformer-base-4096')
longformer.trainable = False

Downloading:   0%|          | 0.00/729M [00:00<?, ?B/s]

Some layers from the model checkpoint at allenai/longformer-base-4096 were not used when initializing TFLongformerModel: ['lm_head']
- This IS expected if you are initializing TFLongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFLongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFLongformerModel were initialized from the model checkpoint at allenai/longformer-base-4096.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFLongformerModel for predictions without further training.


Our model will take in a document and output the class probabilities for each token. 

We build a neural network in keras which takes in a tokenized document along with its attention mask which tells the neural network which tokens to pay attention to. The first layer is the pretrained longformer. The second is a dense layer with 512 nodes. The last layer is a classification layer with 8 nodes per token which represent the class probabilities for each discourse type.

In [ ]:
def build_model():

  token_ids = tf.keras.layers.Input(shape=(max_length,), name='token_ids', dtype=tf.int32)
  attention_mask = tf.keras.layers.Input(shape=(max_length,), name='attention_mask', dtype=tf.int32)


  x = longformer([token_ids, attention_mask])
  y = tf.keras.layers.Dense(512, activation='relu')(x[0])
  w = tf.keras.layers.Dense(8, activation='softmax', dtype='float32')(y)

  local_model = tf.keras.Model(inputs=[token_ids,attention_mask], outputs=w)
  local_model.compile(optimizer=tf.keras.optimizers.Adam(),
                loss=tf.keras.losses.CategoricalCrossentropy())
  return local_model

We set the max number of tokens to 1000. The tokenizer will either truncate or pad each document to 1000 tokens.

In [ ]:
max_length=1000
model1 = build_model()



In [ ]:
model1.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 token_ids (InputLayer)         [(None, 1000)]       0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 1000)]       0           []                               
                                                                                                  
 tf_longformer_model (TFLongfor  TFLongformerBaseMod  148659456  ['token_ids[0][0]',              
 merModel)                      elOutputWithPooling               'attention_mask[0][0]']         
                                (last_hidden_state=                                               
                                (None, 1000, 768),                                            



We perform a train test split on the set of documents.



In [ ]:
train, test = train_test_split(documents,
                               shuffle=True,
                               test_size=.2,
                               random_state=42)

Here we tokenize the training data in preparation for training the model.

In [ ]:

token_ids = np.zeros((len(train),1000))
attention_mask = np.zeros((len(train),1000))


for doc_index, doc in enumerate(train):
  if doc_index%100==0: print(doc_index)
  get_document_token_labels(doc)
  tokens = doc_tokens[doc]
  tokens_tf = [tf.convert_to_tensor(np.asarray(tokens['input_ids'])),tf.convert_to_tensor(np.asarray(tokens['attention_mask'])), tf.convert_to_tensor(np.asarray(tokens['token_type_ids']))]
  
  token_ids[doc_index, ] = tokens_tf[0]
  attention_mask[doc_index, ] = tokens_tf[1]

  
  



0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400


KeyboardInterrupt: ignored

Here we convert the training labels to a format with which we can train our model.

In [ ]:
labels = np.zeros((len(train), 1000, 8))

for doc_index, doc in enumerate(train):
  if doc_index%1000==0: print(doc_index)
  
  labels[doc_index, : , 0:7]=doc_labels[doc]
  ## make empty label
  for token_index, token_labels in enumerate(doc_labels[doc]):
    if np.sum(token_labels)==0:
      labels[doc_index, token_index, 7]=1




Since tokenization takes some time, here we save/load the relevant arrays.

In [ ]:
#np.save('/content/drive/MyDrive/token_ids', token_ids)
#np.save('/content/drive/MyDrive/mask', attention_mask)
#np.save('/content/drive/MyDrive/labels', labels)

token_ids = np.load('/content/drive/MyDrive/token_ids.npy')
attention_mask = np.load('/content/drive/MyDrive/mask.npy')
labels = np.load('/content/drive/MyDrive/labels.npy')


Here we fit the model using the cross categorical entropy loss function over two epochs.

In [ ]:
tf.config.run_functions_eagerly(True)

#model1.fit(x=[token_ids,attention_mask],
   #       y=labels,
 #         batch_size=1,
  #        verbose=1,
  ##        epochs=1)

Here we load the model weights after training.

In [ ]:
#model1.save('/content/drive/MyDrive/model1')


max_length=1000
model1 = build_model()

model1.load_weights('/content/drive/MyDrive/weights')

Here we tokenize the validation data.

In [ ]:
token_ids_validation = np.zeros((len(test),1000))
attention_mask_validation = np.zeros((len(test),1000))


for doc_index, doc in enumerate(test[:5]):
  if doc_index%100==0: print(doc_index)
  get_document_token_labels(doc)
  tokens = doc_tokens[doc]
  tokens_tf = [tf.convert_to_tensor(np.asarray(tokens['input_ids'])),tf.convert_to_tensor(np.asarray(tokens['attention_mask'])), tf.convert_to_tensor(np.asarray(tokens['token_type_ids']))]
  
  token_ids_validation[doc_index, ] = tokens_tf[0]
  attention_mask_validation[doc_index, ] = tokens_tf[1]


In [ ]:
discourse_list= ['Lead', 'Position', 'Evidence', 'Claim', 'Concluding Statement', 'Counterclaim', 'Rebuttal', 'Nothing']

In [ ]:
for token in model1([token_ids_validation[0:1], attention_mask_validation[0:1]]).numpy()[0]:
  print(discourse_list[np.argmax(token)])

Since our network outputs class probabilities for each token individually, the raw outputs are fairly discontinuous. To deal with this, we count the predictions for each token in a given sentence and label the sentence with the most predicted class. To do this however, we have to determine which token belongs to which sentence, which we do below.

In [ ]:
def extract_sentences(tokens):
  token_index = 0
  sentence_list = []

  token_ids = tokens['input_ids']
  while token_index < max_length:
    new_sentence = []
    within_sentence = True

    while within_sentence and token_index < max_length:
      if token_ids[token_index]!=4:
        new_sentence.append(token_index)
      else:
        within_sentence = False
      token_index += 1
    
    sentence_list.append(new_sentence)

  return sentence_list

def extract_sentences_characters(tokens):
  #returns a list of tuples corresponding to character beginning and ends 
  # of sentences
  offset_mapping = tokens.offset_mapping
  sentence_list = extract_sentences(tokens)

  sentence_character_list = []

  for sentence in sentence_list:
    begin = offset_mapping[sentence[0]][0]
    end = offset_mapping[sentence[-1]][1]
  
    sentence_character_list.append((begin, end))
  
  return sentence_character_list



def predict(string):
 token_ids_list = np.zeros((2,1000))
 attention_mask_list = np.zeros((2,1000))  
  
 tokens =  get_tokens(string)
 tokens_tf = [tf.convert_to_tensor(np.asarray(tokens['input_ids'])),tf.convert_to_tensor(np.asarray(tokens['attention_mask'])), tf.convert_to_tensor(np.asarray(tokens['token_type_ids']))]
  
 token_ids_list[0, ] = tokens_tf[0]
 attention_mask_list[0, ] = tokens_tf[1]

 return model1([token_ids_list[0:1], attention_mask_list[0:1]])


The function below gives the predictions for a arbitrary input string.

In [ ]:
def get_string_predictions(string):
  sentence_predictions = aggregate_sentences(string)
  string_predictions = []
  within_sentence = False

  for index, word in enumerate(string.split()):
    if not within_sentence:
      prediction_string =''
      within_sentence = True
    prediction_string += ' ' + str(index)

    if word[-1]=='.':
      within_sentence = False
      string_predictions.append(prediction_string[1:])

  ## merge strings according to certain rules
  merged_strings = []

  merge = True
  new_string = ''
  for index, string in enumerate(string_predictions):
    if merge:
      new_string += string
    if index+1 < len(sentence_predictions):
      merge = sentence_predictions[index+1]==sentence_predictions[index]

      if not merge:
        merged_strings.append(new_string)
        new_string =' '
        merge = True
    else:
      merged_strings.append(new_string)

  return merged_strings

def remove_adjacent(list):
  new_list= []
  for item in list:
    if len(new_list):
      if new_list[-1] != item:
        new_list.append(item)
    else: new_list.append(item)        
  return new_list


The function below gives sentence predictions according to the outputs of the neural network and certain rules. Since the neural network tends to confuse the discourse type 'position' with 'concluding statement' for example, we give extra weights to the concluding statement class once a position is already stated.



In [ ]:
### can try to come up with a prior list depending on sentence position

##### aggregate sentences first
def aggregate_sentences(string):
  tokens = get_tokens(string)
  sentence_predictions = []
  sentences = extract_sentences(tokens)
  input_ids = tokens['input_ids']
  attention_mask = tokens['attention_mask']
  token_predictions = predict(string)
  weight_matrix = np.diag([1,1,1,1,1,1,1,1])
  for sentence_number, sentence in enumerate(sentences):
      overall_sentence_counts = np.array([0,0,0,0,0,0,0,0])
      lead_is_over = ('Lead' in sentence_predictions) and ('Claim' in sentence_predictions or 'Evidence' in sentence_predictions or 'Position' in sentence_predictions )
      position_is_stated = 'Position' in sentence_predictions

      if lead_is_over:
        weight_matrix = np.diag([.1,30,.05,3,1,20,20,1])
      if not position_is_stated:
        weight_matrix = np.add(np.diag([1,10,.05,3,1,20,40,1]),np.matrix([[0,0,0,0,0,0,0,0],
                                                                       [0,0,0,10,10,0,0,0],
                                                                       [0,0,0,0,0,0,0,0],
                                                                       [0,0,0,0,0,0,0,0],
                                                                       [0,0,0,0,0,0,0,0],
                                                                       [0,0,0,0,0,0,0,0],
                                                                       [0,0,0,0,0,0,0,0],
                                                                       [0,0,0,0,0,0,0,0]]))
      if position_is_stated:
        weight_matrix = np.add(np.diag([.3,.3,.1,7,3,20,40,.3]),np.matrix([[0,0,0,0,0,0,0,0],
                                                                       [0,0,0,0,0,0,0,0],
                                                                       [0,0,0,0,0,0,0,0],
                                                                       [0,0,0,0,0,0,0,0],
                                                                       [0,sentence_number/10 ,0,0,0,0,0,0],
                                                                       [0,0,0,0,0,0,0,0],
                                                                       [0,0,0,0,0,0,0,0],
                                                                       [0,0,0,0,0,0,0,0]]))
      for token in sentence:

        token_prediction = np.argmax(token_predictions[0][token])
        overall_sentence_counts[token_prediction] += 1
      weighted_counts = np.matmul(weight_matrix, overall_sentence_counts)
 
      prediction = np.argmax(weighted_counts)
      sentence_predictions.append(discourse_list[prediction])
  return sentence_predictions




## use this to optimize prior
def multiply_by_prior(sentence_predictions, prior_list):
  new_predictions = []
  for index, sentence in sentence_predictions:
    prediction_probabilities = np.multiply(prior_list[index], overall_sentence)
  new_predictions.append(prediction_probabilities)

  return new_predictions

def give_sentence_predictions(doc_number, prior_list):
  sentence_probabilities = multiply_by_prior(aggregate_sentences(doc_number), prior_list)
  sentence_labels = []
  for index, probabilities in enumerate(sentence_probabilities):
    sentence_labels.append(np.argmax(probabilities))

def constant_weighted_sentence_predictions(doc_number, prior_list):
  sentence_predictions = []
  sentences = extract_sentences(doc_tokens[doc_number])
  input_ids = doc_tokens[doc_number]['input_ids']
  attention_mask = doc_tokens[doc_number]['attention_mask']
  token_predictions = predict(doc_number)

  for sentence_number, sentence in enumerate(sentences):
      overall_sentence = np.array([0,0,0,0,0,0,0,0])
      for token_index, token in enumerate(sentence):
        overall_sentence = np.add(overall_sentence, token_predictions[0][token_index])
      prediction_probabilities = np.multiply(prior_list[sentence_number], overall_sentence)
      prediction = np.argmax(prediction_probabilities)

  return sentence_predictions


Here we write some nearest neighbors functions that predict token classes while taking into account the predicted classes of nearby tokens. 

In [ ]:
## some nearest_neighbors helper functions
def naive_nearest_neighbors(predictions, k):
  np_predictions = predictions.numpy()
  # for loop through each document

  nearest_neighbors_predictions = np.zeros((len(np_predictions), max_length))
  for doc_index, docs in enumerate(np_predictions):
    for token_index, token in enumerate(docs[: k//2+1]):
      k_neighbors = docs[token_index: token_index + k]
      sum_of_k_neighbors = np.sum(k_neighbors, axis=0)
      nearest_neighbors_predictions[doc_index, token_index] = np.argmax(sum_of_k_neighbors)
    for token_index, token in enumerate(docs[k//2+1 : - k//2]):
      k_neighbors = docs[token_index - k//2 : token_index + k//2]
      sum_of_k_neighbors = np.sum(k_neighbors, axis=0)
      nearest_neighbors_predictions[doc_index, token_index] = np.argmax(sum_of_k_neighbors)
    for token_index, token in enumerate(docs[ - k//2 : ]):
      k_neighbors = docs[token_index - k : ]
      sum_of_k_neighbors = np.sum(k_neighbors, axis=0)
      nearest_neighbors_predictions[doc_index, token_index] = np.argmax(sum_of_k_neighbors)
  return nearest_neighbors_predictions

##below will be used as a last layer in the neural network, instead of computing the most likely class given the neighbors we compute the probabilities of each class by averaging the neighbors

## should optimize so that it doesn't pass periods or commas

def naive_nearest_neighbors_probabilities(predictions, k):
  np_predictions = predictions.numpy()
  # for loop through each document

  nearest_neighbors_predictions = np.zeros((len(np_predictions), max_length))
  for doc_index, docs in enumerate(np_predictions):
    for token_index, token in enumerate(docs[: k//2+1]):
      k_neighbors = docs[token_index: token_index + k]
      sum_of_k_neighbors = np.sum(k_neighbors, axis=0)/len(k_neighbors)
    for token_index, token in enumerate(docs[k//2+1 : - k//2]):
      k_neighbors = docs[token_index - k//2 : token_index + k//2]
      sum_of_k_neighbors = np.sum(k_neighbors, axis=0)//len(k_neighbors)
    for token_index, token in enumerate(docs[ - k//2 : ]):
      k_neighbors = docs[token_index - k : ]
      sum_of_k_neighbors = np.sum(k_neighbors, axis=0)/len(k_neighbors)
  return nearest_neighbors_predictions


def k_window(input_ids, token_index, k):
  if k==0:
    return token_index  
  
  for n in range(k):  
    shift_factor = n*((-1)**n)
    left = token_index - k//2 + shift_factor
    right =  token_index + k//2 + shift_factor
    ## make sure there is no period in the window
    ## need to iterate over input_ids which is a keras tensor
    
    candidate = [input_ids[i] for i in range(left,right)]

    for i in candidate:
      if i== 4: continue
    else: 
    # return the first window that doesn't have a period
      return range(left, right+1)

  ## if there is no k_window decrease k and search again   
  return k_window(input_ids, token_index, k-1)

def show_nn_predictions(doc_number, k): 
  i = 0
  for x in k_windows_layer(token_ids_validation[doc_number:doc_number+1], model1([token_ids_validation[doc_number:doc_number+1], attention_mask_validation[doc_number:doc_number+1]]),10)[0]:
    print(discourse_list[int(np.argmax(x))], tokenizer.decode(doc_tokens[test[doc_number]]['input_ids'][i]))
    i += 1
def show_predictions(doc_number, k): 
  i = 0
  for x in model1([token_ids_validation[doc_number:doc_number+1], attention_mask_validation[doc_number:doc_number+1]])[0]:
    print(discourse_list[int(np.argmax(x))], tokenizer.decode(doc_tokens[test[doc_number]]['input_ids'][i]))
    i += 1


Here we write a function that creates a dataframe of predictions for a given document in the required format for the kaggle competition.

In [ ]:
def create_columns(document_list):
  prediction_string_column = []
  document_id_column = []
  discourse_type_column = []

  for doc in document_list:
    with open('train/'+ doc +'.txt') as f:
      lines = f.read()
      prediction_strings = get_string_predictions(lines)
      predictions = remove_adjacent(aggregate_sentences(lines))
      for sentence_number, prediction_string in enumerate(prediction_strings):
        if sentence_number < len(predictions):
          prediction_string_column.append(prediction_string)
          discourse_type_column.append(predictions[sentence_number])
          document_id_column.append(doc)
      
      ##merge adjacent rows of the same type
  return pd.DataFrame({'id': document_id_column, 'class': discourse_type_column, 'predictionstring': prediction_string_column})

    

In [ ]:
create_columns(train[0:10])

['0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33', '34 35 36 37 38 39 40 41 42 43 44 45 46 47 48', '49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73', '74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93', '94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109', '110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130', '131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162', '163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210', '211 212 213 214 215 216 217 218 219 220 221 222', '223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250', '251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 26

,id,class,predictionstring
0,F56828B65A0B,Lead,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
1,F56828B65A0B,Position,34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
2,F56828B65A0B,Concluding Statement,49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 ...
3,F56828B65A0B,Claim,74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 ...
4,F56828B65A0B,Evidence,110 111 112 113 114 115 116 117 118 119 120 1...
...,...,...,...
107,D337DC4042D2,Evidence,622 623 624 625 626 627 628 629 630 631 632 6...
108,D337DC4042D2,Counterclaim,786 787 788 789 790 791 792 793 794 795 796 7...
109,D337DC4042D2,Concluding Statement,812 813 814 815 816 817 818 819 820 821 822 8...
110,D337DC4042D2,Counterclaim,843 844 845 846 847 848 849 850 851 852 853 854


Below is some code from https://www.kaggle.com/code/robikscube/student-writing-competition-twitch-stream?scriptVersionId=83303421 which calculates the macro F1 score for the predictions across all classes.



In [ ]:
## f1 metric from https://www.kaggle.com/code/robikscube/student-writing-competition-twitch-stream?scriptVersionId=83303421

def calc_overlap(row):
    """
    Calculates the overlap between prediction and
    ground truth and overlap percentages used for determining
    true positives.
    """
    set_pred = set(row.predictionstring_pred.split(" "))
    set_gt = set(row.predictionstring_gt.split(" "))
    # Length of each and intersection
    len_gt = len(set_gt)
    len_pred = len(set_pred)
    inter = len(set_gt.intersection(set_pred))
    overlap_1 = inter / len_gt
    overlap_2 = inter / len_pred
    return [overlap_1, overlap_2]


def score_feedback_comp_micro(pred_df, gt_df):
    """
    A function that scores for the kaggle
        Student Writing Competition

    Uses the steps in the evaluation page here:
        https://www.kaggle.com/c/feedback-prize-2021/overview/evaluation
    """
    gt_df = (
        gt_df[["id", "discourse_type", "predictionstring"]]
        .reset_index(drop=True)
        .copy()
    )
    pred_df = pred_df[["id", "class", "predictionstring"]].reset_index(drop=True).copy()
    pred_df["pred_id"] = pred_df.index
    gt_df["gt_id"] = gt_df.index
    # Step 1. all ground truths and predictions for a given class are compared.
    joined = pred_df.merge(
        gt_df,
        left_on=["id", "class"],
        right_on=["id", "discourse_type"],
        how="outer",
        suffixes=("_pred", "_gt"),
    )
    joined["predictionstring_gt"] = joined["predictionstring_gt"].fillna(" ")
    joined["predictionstring_pred"] = joined["predictionstring_pred"].fillna(" ")

    joined["overlaps"] = joined.apply(calc_overlap, axis=1)

    # 2. If the overlap between the ground truth and prediction is >= 0.5,
    # and the overlap between the prediction and the ground truth >= 0.5,
    # the prediction is a match and considered a true positive.
    # If multiple matches exist, the match with the highest pair of overlaps is taken.
    joined["overlap1"] = joined["overlaps"].apply(lambda x: eval(str(x))[0])
    joined["overlap2"] = joined["overlaps"].apply(lambda x: eval(str(x))[1])

    joined["potential_TP"] = (joined["overlap1"] >= 0.5) & (joined["overlap2"] >= 0.5)
    joined["max_overlap"] = joined[["overlap1", "overlap2"]].max(axis=1)
    tp_pred_ids = (
        joined.query("potential_TP")
        .sort_values("max_overlap", ascending=False)
        .groupby(["id", "predictionstring_gt"])
        .first()["pred_id"]
        .values
    )

    # 3. Any unmatched ground truths are false negatives
    # and any unmatched predictions are false positives.
    fp_pred_ids = [p for p in joined["pred_id"].unique() if p not in tp_pred_ids]

    matched_gt_ids = joined.query("potential_TP")["gt_id"].unique()
    unmatched_gt_ids = [c for c in joined["gt_id"].unique() if c not in matched_gt_ids]

    # Get numbers of each type
    TP = len(tp_pred_ids)
    FP = len(fp_pred_ids)
    FN = len(unmatched_gt_ids)
    # calc microf1
    my_f1_score = TP / (TP + 0.5 * (FP + FN))
    return my_f1_score


def score_feedback_comp(pred_df, gt_df, return_class_scores=False):
    class_scores = {}
    pred_df = pred_df[["id", "class", "predictionstring"]].reset_index(drop=True).copy()
    for discourse_type, gt_subset in gt_df.groupby("discourse_type"):
        pred_subset = (
            pred_df.loc[pred_df["class"] == discourse_type]
            .reset_index(drop=True)
            .copy()
        )
        class_score = score_feedback_comp_micro(pred_subset, gt_subset)
        class_scores[discourse_type] = class_score
    f1 = np.mean([v for v in class_scores.values()])
    if return_class_scores:
        return f1, class_scores
    return f1

Below is the macro F1 score across all classes for the test data, along with the individual F1 scores for each class.

In [ ]:
score_feedback_comp(create_columns(data[-1000:].id.unique().tolist()), data[-1000:], return_class_scores=True)

['0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33', '34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88', '89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120', '121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165', '166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225', '226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258', '259 260 261 262 263 264 265 266 267 268 269 270', '271

(0.22778787468924752,
 {'Claim': 0.0843585237258348,
  'Concluding Statement': 0.3206997084548105,
  'Counterclaim': 0.0,
  'Evidence': 0.5606299212598426,
  'Lead': 0.41830065359477125,
  'Position': 0.21052631578947367,
  'Rebuttal': 0.0})

Here we write a function that prints the color coded predictions for a given string using the sty package.

In [ ]:
!pip install sty
import sty
from sty import bg, rs 
def print_predictions(string):
  prediction_strings = get_string_predictions(lines)
  predictions = remove_adjacent(aggregate_sentences(lines))

  split = string.split()
  prediction_count = 0

  new_color = True

  new_string = ''
  
  for index, word in enumerate(split):
    if new_color:
      if prediction_count<len(predictions):
        new_string += bg(int(color_dict[predictions[prediction_count]]))
      new_color = False
    new_string += ' ' + word

    if (index + 1) % 10 == 0:
      new_string += '\n'
    int_string = [int(i) for i in prediction_strings[prediction_count].split()]
    if index == int_string[-1]:
      new_color = True
      new_string += rs.all 
      prediction_count += 1
  return new_string
color_dict = {'Nothing': 15 , 'Lead': 14, 'Claim': 13, 'Counterclaim': 12, 'Rebuttal': 11, "Evidence": 10, 'Position': 9, 'Concluding Statement': 8}






Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Some predictions on one of the training essays.

In [ ]:
print('Color codes:' )
for key in color_dict.keys():
  print(bg(color_dict[key]) + key)
print('--------------')
with open('/content/train/' +train[2] + '.txt') as f:
  lines = f.read()
print(print_predictions(lines))



Color codes:
Nothing
Lead
Claim
Counterclaim
Rebuttal
Evidence
Position
Concluding Statement
--------------
 When people ask for advice, they often talk to more
 than one person. For example, when you are in need
 of someone in a difficult situation, you probably talk to
 more than one person to see who has the best
 answer to your problem. I think seeking multiple opinions is
 better because it gives you a choice on what you
 want to do,you have another piece of advice that you
 can rely on, and it can help you see that
 the first opinion does not make any sense First, if
 you get more than one opinion you get to choose
 what you feel is the best option for your problem.
 I remember when i was young i only ever asked
 one person for advice and that was my mom. But
 she didnt always have the best advice and i paid
 the price for that one time. I was really small
 then and i asked my mom if i could play
 tackle football and she said that i could. but I
 was way too small back then so i 

In [ ]:
!pip install streamlit -q

In [ ]:
%%writefile app.py
import streamlit as st
st.title('Essay Annotater')
st.header('Some context.')

user_input = st.text_area("Paste your essay here.")

st.text(print_predictions(user_input))

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501

Color codes:
Nothing
Lead
Claim
Counterclaim
Rebuttal
Evidence
Position
Concluding Statement
--------------
 When people ask for advice, they often talk to more
 than one person. For example, when you are in need
 of someone in a difficult situation, you probably talk to
 more than one person to see who has the best
 answer to your problem. I think seeking multiple opinions is
 better because it gives you a choice on what you
 want to do,you have another piece of advice that you
 can rely on, and it can help you see that
 the first opinion does not make any sense First, if
 you get more than one opinion you get to choose
 what you feel is the best option for your problem.
 I remember when i was young i only ever asked
 one person for advice and that was my mom. But
 she didnt always have the best advice and i paid
 the price for that one time. I was really small
 then and i asked my mom if i could play
 tackle football and she said that i could. but I
 was way too small back then so i broke my
 leg on my first play. but if i asked for
 another opinion i probably would've played one more season of
 flag football. Next, if you get more than one opinion,
 you have another piece of advice that you can rely
 on. If you have a opinion that sounds really bad
 but you have another opinion that sounds really good, you
 go with the good advice. but if you never asked
 for another opinion, and you just went with the first,
 really bad option, it could mess up something for you.
 say you were going to marry a girl but you
 mom said not to because she isnt rich enough, but
 you really love her so you ask your dad and
 he says marry her, you'd probably go with your dads
 opinion because you love her so much. but if you
 never got your dads opinion, you wouldnt be marrying her
 because your mom does not like her. Finally, if you
 get more than one opinion, it could help make you
 see that the first opinion does not make any sense
 at all. If you're a student having trouble with a
 subject, and you ask another student for advice on what
 to do, and you're not sure that what they said
 is true and you go ask the teacher and what
 the teacher said to do makes way more sense, you
 go with what the teacher said. But again, if you
 had'nt asked for that second opinion, you would've gotton all
 of that subject wrong. When people ask for advice they
 often talk to more than one person. I think seeking
 multiple opinions i better because it gives you a choice
 on what you want to do, you have another piece
 of advice you can rely on, and it can help
 you see that the other opinion might not make sense.